# Heat exchanger model calibration and validation

TODO: Use pre-processing and data conditioning functions from `models_psa.utils`

### Calibrated parameters


### Version 20240306

- UA: 13536.596 W/K

### Version 202309

- UA: 2160 W/K

In [1]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations.utils import rename_signal_ids_to_var_ids
from phd_visualizations.utils.units import unit_conversion
from solarMED_modeling.validation import within_range_or_nan_or_max, within_range_or_zero_or_max
from solarMED_modeling.curve_fitting import evaluate_fit

import sys
sys.path.append('..')

%load_ext autoreload

resample_figures = False

data_path = Path(f'{os.getenv("HOME")}/development_psa/models_psa/data')

# Visualize result so far
with open( data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)
    
# Available data to test
# data/calibration/20230807_aquasol.csv
# data/calibration/20230707_20230710_datos_tanques.csv
# Nextcloud/Juanmi_MED_PSA/EURECAT/data/20231030_solarMED.csv

sample_rate = '10s'
sample_rate_numeric = int(sample_rate[:-1])

In [170]:
# Load already procesed dev csv
# df = pd.read_csv('../data/dev.csv', parse_dates=['time'], index_col='time')

# Load raw data and perform processing
data_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/data')
filename_process_data = '20230807_aquasol.csv'

df = pd.read_csv(data_path / filename_process_data, parse_dates=True, index_col='TimeStamp')
# Rename index column to "time"
df.index.names = ['time']
# Set UTC timezone
df = df.tz_localize('UTC')

# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)
# Convert units to model units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')


df["epsilon"] = np.nan
date_str = df.index[0].strftime("%Y-%m-%d")

In [171]:
# Once completed, this should be moved to models_psa/heat_exchanger.py

def estimate_flow_secondary(Tp_in:float, Ts_in:float, qp:float, Tp_out: float, Ts_out: float) -> float:
    
    w_props_Tp_in = w_props(P=0.16, T=(Tp_in+Tp_out)/2 + 273.15)
    w_props_Ts_in = w_props(P=0.16, T=(Ts_in+Ts_out)/2 + 273.15)
    
    cp_Tp_in = w_props_Tp_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
    cp_Ts_in = w_props_Ts_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
    
    qs = qp * (cp_Tp_in * (Tp_in - Tp_out)) / (cp_Ts_in * (Ts_out - Ts_in))
    
    return np.max([qs, 0])

In [172]:
from solarMED_modeling.power_consumption import actuator_coefficients

df['qhx_s_fit'] = evaluate_fit(df['UK-SF-P001-fq'], fit_type='quadratic_curve', params=actuator_coefficients['FTSF001_calibration'])
# Filter out negative values
df.loc[df['qhx_s_fit'] < 0, 'qhx_s_fit'] = 0


idx_start = 900
idx_end = len(df)-1200

df['qhx_s2_estimated'] = np.nan
for idx in range(idx_start,idx_end):
    df.loc[df.index[idx], 'qhx_s2_estimated'] = estimate_flow_secondary(df.loc[df.index[idx], 'Thx_p_in'], df.loc[df.index[idx], 'Thx_s_in'], df.loc[df.index[idx], 'qhx_p'], df.loc[df.index[idx], 'Thx_p_out'], df.loc[df.index[idx], 'Thx_s_out'])
    

In [2]:
plt_config = {
  # General plot attributes
  "title": "Heat exchanger",
  "subtitle": "model validation",
  "height": 1000,
  "width": 1000,
  "margin": {
    "l":20,
    "r":100,
    "t":100,
    "b":20,
    "pad":5
  },
  "vertical_spacing": 0.03,
  "xdomain": [0, 0.85],
  "arrow_xrel_pos": 60, # seconds

  "plots": {
    
    "heat_exchanger_flows": {
        "title": "<b>Flows</b>",
        "row_height": 1,
        "bg_color": "bg_gray", # bg gray
        "ylabels_left": ["m<sup>3</sup>/h"],
        "ylabels_right": ["m<sup>3</sup>/h"],
        "ylims_left": 'manual',
        
        "traces_left": [
            {"var_id": "qhx_p", "color": "plotly_red", "axis_arrow": True},
        ],
        "traces_right": [
            {"var_id": "qhx_s", "color": "plotly_blue", "axis_arrow": True},
            {"var_id": "qhx_s_fit", "color": "plotly_blue", "axis_arrow": True, "dash": "longdash"},
            {"var_id": "qhx_s2_estimated", "color": "plotly_blue", "axis_arrow": True, "dash": "dashdot"},
        ]
    },

    "heat_exchanger_temperatures": {
        "title": "<b>Temperatures</b>",
        "row_height": 1,
        "bg_color": "bg_gray", # bg gray
        "ylabels_left": ["⁰C"],
        "ylims_left": "manual",
        
        "traces_left": [
            {"var_id": "Thx_p_in", "color": "plotly_red",}, # "fill": "tonexty", "fill_pattern": "\\",
            {"var_id": "Thx_p_out", "color": "plotly_red", "dash": "longdash"},
            {"var_id": "Thx_s_out", "color": "plotly_blue", "dash": "longdash"}, # "fill": "tonexty", "fill_pattern": "/",
            {"var_id": "Thx_s_in", "color": "plotly_blue"},
        ]
    },
      
    "effectiveness": {
        "title": "<b>Effectiveness - NTU</b>",
        "row_height": 0.5,
        "bg_color": "plotly_yellow", # bg gray
        "ylabels_left": ["-"],
        "ylims_left": [0,1],
        
        "traces_left": [
            {"var_id": "epsilon", "color": "plotly_green", "name": "ε"},
        ]
    },
    
  }

}

In [3]:
# # Save to json
# import json
# 
# with open( Path("../data") / 'plt_config_heat_exchanger.json', 'w') as f:
#     json.dump(plt_config, f, indent=4)

In [174]:
%autoreload 2

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'heat_exhanger_validation_{df.index[0].strftime("%Y%m%d")}')
)

In [175]:
# Remove from visualization
# del plt_config['plots']['heat_exchanger_flows']['traces_right'][-1]

In [166]:
# Since flow signal is fucked up, replace it with the estimated one


# df.rename(columns={'qhx_s': 'qhx_s_original', 'qhx_s2_estimated': 'qhx_s'}, inplace=True)
# Keep a copy
# df['qhx_s2_estimated'] = df['qhx_s']


# plt_config['plots']['heat_exchanger_flows']['traces_right'].append(
#     {"var_id": "qhx_s_original", "color": "plotly_blue", "axis_arrow": True},
# )


In [110]:
# save_figure(
#     figure_name=f"heat_exchanger_visualization_{df.index[0].strftime('%Y%m%d')}", 
#     figure_path=Path("/home/jmserrano/Downloads"),
#     fig=fig, formats=('png', ), 
#     width=fig.layout.width, height=fig.layout.height, scale=2
# )

In [176]:
from phd_visualizations.heat_exchanger import steady_state_viz
# 
%autoreload 2

date_idx = [f'{date_str} 10:30:00', f'{date_str} 12:00:00', f'{date_str} 12:30:00', f'{date_str} 14:00:00',]

fig = steady_state_viz(df, date_idx, include_limits=True)

fig.show()

In [177]:
# Save figure
save_figure(
    figure_name=f"heat_exchanger_validation", 
    figure_path=Path("/home/jmserrano/Downloads"),
    fig=fig, formats=('svg', 'png'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

In [142]:
# Once completed, this should be moved to models_psa/heat_exchanger.py

from iapws import IAPWS97 as w_props
import math
from typing import Literal

def heat_exchanger_model(Tp_in:float, Ts_in:float, qp:float, qs:float, Tamb:float, UA:float=28000, H:float=0,
                         log: bool = True, hex_type:Literal['counter_flow', ] = 'counter_flow', return_epsilon: bool = False, epsilon: float = None):  # eta_p, eta_s):

    """Counter-flow heat exchanger steady state model.
    
    Based on the effectiveness-NTU method [2] - Chapter Heat exchangers 11-5:
    
    ΔTa: Temperature difference between primary circuit inlet and secondary circuit outlet
    ΔTb: Temperature difference between primary circuit outlet and secondary circuit inlet
    
    `p` references the primary circuit, usually the hot side, unless the heat exchanger is inverted.
    `s` references the secondary circuit, usually the cold side, unless the heat exchanger is inverted.
    `Qdot` is the heat transfer rate
    `C` is the capacity ratio, defined as the ratio of the heat capacities of the two fluids, C = Cmin/Cmax
    
    To avoid confussion, whichever the heat exchange direction is, the hotter side will be referenced as `h` and the colder side as `c`.
    
   T|  Tp,in
    |   ---->
    |    .   \---->         Tp,out
    |   ΔTa       \----------->
    |    .                ΔTb
    |    <---              .
    |       \<----------------<
    |    Ts,out               Ts,in
    |_______________________________
                                   z
                                   
    Limitations (from [2]):
    - It has been assumed that the rate of change for the temperature of both fluids is proportional to the temperature difference; this assumption is valid for fluids with a constant specific heat, which is a good description of fluids changing temperature over a relatively small range. However, if the specific heat changes, the LMTD approach will no longer be accurate.
    - A particular case for the LMTD are condensers and reboilers, where the latent heat associated to phase change is a special case of the hypothesis. For a condenser, the hot fluid inlet temperature is then equivalent to the hot fluid exit temperature.
    - It has also been assumed that the heat transfer coefficient (U) is constant, and not a function of temperature. If this is not the case, the LMTD approach will again be less valid
    - The LMTD is a steady-state concept, and cannot be used in dynamic analyses. In particular, if the LMTD were to be applied on a transient in which, for a brief time, the temperature difference had different signs on the two sides of the exchanger, the argument to the logarithm function would be negative, which is not allowable.
    - No phase change during heat transfer
    - Changes in kinetic energy and potential energy are neglected

    [1] W. M. Kays and A. L. London, Compact heat exchangers: A summary of basic heat transfer and flow friction design data. McGraw-Hill, 1958. [Online]. Available: https://books.google.com.br/books?id=-tpSAAAAMAAJ
    
    [2] Y. A. Çengel and A. J. Ghajar, Heat and mass transfer: fundamentals & applications, Fifth edition. New York, NY: McGraw Hill Education, 2015.

    Args:
        Tp_in (float): Primary circuit inlet temperature [C]
        Ts_in (float): Secondary circuit inlet temperature [C]
        qp (float): Primary circuit volumetric flow rate [m^3/h]
        qs (float): Secondary circuit volumetric flow rate [m^3/h]
        UA (float, optional): Heat transfer coefficient multiplied by the exchange surface area [W·ºC^-1]. Defaults to 28000.

    Returns:
        Tp_out: Primary circuit outlet temperature [C]
        Ts_out: Secondary circuit outlet temperature [C]
    """

    # TODO: Add option to simplify model by using constant water properties

    if hex_type != 'counter_flow':
        raise ValueError('Only counter-flow heat exchangers are supported')

    inverted_hex = False

    w_props_Tp_in = w_props(P=0.16, T=Tp_in + 273.15)
    w_props_Ts_in = w_props(P=0.16, T=Ts_in + 273.15)
    cp_Tp_in = w_props_Tp_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
    cp_Ts_in = w_props_Ts_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]

    mp = qp / 3600 * w_props_Tp_in.rho  # rho [kg/m³] # Convertir m^3/s a kg/s
    ms = qs / 3600 * w_props_Ts_in.rho  # rho [kg/m³] # Convertir m^3/s a kg/s
    
    Cp = mp*cp_Tp_in
    Cs = ms*cp_Ts_in
    Cmin = np.min([Cp, Cs])
    Cmax = np.max([Cp, Cs])

    # mcp_min = min([mp*cp_Tp_in, ms*cp_Ts_in])
    # mcp_max = max([mp*cp_Tp_in, ms*cp_Ts_in])

    # theta = UA*(1/mcp_max-1/mcp_min)

    # eta_p = (1-math.e**theta)/( 1-math.e**theta*(mcp_min/mcp_max) )
    # eta_s = mp*cp_Tp_in/(ms*cp_Ts_in)

    # Tp_out = Tp_in - eta_p*(mcp_min)/(mp*cp_Tp_in)*(Tp_in-Ts_in) - H*(Tp_in-Tamb) # ºC
    # Ts_out = Ts_in + eta_s*(Tp_in-Tp_out) # ºC

    if qp < 0.1:
        Tp_out = Tp_in - H * (Tp_in - Tamb) # Just losses to the environment
        if qs < 0.1:
            Ts_out = Ts_in - H * (Ts_in - Tamb) # Just losses to the environment
        else:
            Ts_out = Ts_in
            
        if return_epsilon:
            return Tp_out, Ts_out, 0
        else:
            return Tp_out, Ts_out
    
    if qs < 0.1:
        Ts_out = Ts_in - H * (Ts_in - Tamb) # Just losses to the environment
        if qp < 0.1:
            Tp_out = Tp_in - H * (Tp_in - Tamb) # Just losses to the environment
        else:
            Tp_out = Tp_in

        if return_epsilon:
            return Tp_out, Ts_out, 0
        else:
            return Tp_out, Ts_out
        

    if Tp_in < Ts_in:
        inverted_hex = True
        
        if log: logger.warning('Inverted operation in heat exchanger')
        
        Ch = Cs
        Cc = Cp
        Th_in = Ts_in
        Tc_in = Tp_in
        
    else:
        Ch = Cp
        Cc = Cs
        Th_in = Tp_in
        Tc_in = Ts_in

    # Calculate the effectiveness
    if epsilon is None:
        C = Cmin / Cmax
        NTU = UA / Cmin
        epsilon = ( 1-math.e**(-NTU*(1-C)) ) / ( 1-C*math.e**(-NTU*(1-C)) )

    # Calculate the heat transfer rate
    Qdot_max = Cmin * (Th_in - Tc_in)
    
    # Calculate the outlet temperatures
    # Assume that the losses to the environment are dominated from the inlet hot side temperature (maximun temperature difference)
    Th_out = Th_in - (Qdot_max*epsilon) / (Ch) # - H * (Th_in - Tamb)
    # Tc,out = Tc_in + (Qdot*epsilon) / (Cc) - H * (Tc_in + Qdot,max/Cc - Tamb)
    # Assume that the maximum heat transfer rate is achived to the cold side for the thermal losses (maximun temperature difference)
    Tc_out = Tc_in + (Qdot_max*epsilon) / (Cc) # - H * (Tc_in + Cmin*(Th_in-Tc_in)/Cc - Tamb)

    if inverted_hex:
        Tp_out = Tc_out
        Ts_out = Th_out
        
    else:
        Tp_out = Th_out
        Ts_out = Tc_out

    if return_epsilon:
        return Tp_out, Ts_out, epsilon
    else:
        return Tp_out, Ts_out

def calculate_heat_transfer_effectiveness(Tp_in: float, Tp_out: float, Ts_in: float, Ts_out: float, qp: float, qs: float) -> float:
    """
    Equation (11–33) from [1]

    [1] Y. A. Çengel and A. J. Ghajar, Heat and mass transfer: fundamentals & applications, Fifth edition. New York, NY: McGraw Hill Education, 2015.

    Returns:
        eta: Heat transfer effectiveness

    """

    w_props_Tp_in = w_props(P=0.16, T=Tp_in + 273.15)
    w_props_Ts_in = w_props(P=0.16, T=Ts_in + 273.15)

    cp_Tp_in = w_props_Tp_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
    cp_Ts_in = w_props_Ts_in.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]

    mp = qp / 3600 * w_props_Tp_in.rho  # rho [kg/m³] # Convertir m^3/s a kg/s
    ms = qs / 3600 * w_props_Ts_in.rho  # rho [kg/m³] # Convertir m^3/s a kg/s

    Cmin = np.min([mp*cp_Tp_in, ms*cp_Ts_in])

    # It could be calculated with any, just to disregard specific heat capacity
    if abs(Cmin - mp*cp_Tp_in) < 1e-6:  # Primary circuit is the one with the lowest heat capacity
        epsilon = (Tp_in - Tp_out) / (Tp_in - Ts_in)
    else: # Secondary circuit is the one with the lowest heat capacity
        epsilon = (Ts_out - Ts_in) / (Tp_in - Ts_in)

    return epsilon

ds = df.loc[f'{date_str} 12:00:00']

Tp_in = ds['Thx_p_in']
Ts_in = ds['Thx_s_in']
qp = ds['qhx_p']
qs = ds['qhx_s']
Tamb = ds['Tamb']

logger.info(f'Tp_in: {Tp_in:.2f} C, Ts_in: {Ts_in:.2f} C, qp: {qp:.2f} m³/h, qs: {qs:.2f} m³/h, Tamb: {Tamb:.2f} C')

wprops_p = w_props(P=0.16, T=Tp_in + 273.15)
wprops_s = w_props(P=0.16, T=Ts_in + 273.15)

cp_p = wprops_p.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
cp_s = wprops_s.cp * 1e3  # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]

Cp = qp / 3600 * wprops_p.rho * cp_p # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
Cs = qs / 3600 * wprops_s.rho * cp_s # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]

logger.info(f'Cp: {Cp:.2f} J/K, Cs: {Cs:.2f} J/K, cp_p: {cp_p:.2f} J/K, cp_s: {cp_s:.2f} J/K')

Cmin = np.min([Cp, Cs])

Qmax = Cmin * (Tp_in - Ts_in)

logger.info(f'Qmax: {Qmax:.2f} W')

# In the limit case
Tp_out_min = Tp_in - Qmax / Cp
Ts_out_max = Ts_in + Qmax / Cs

# Calculate power
Phx_p = qp / 3600 * wprops_p.rho * cp_p * (Tp_in - ds["Thx_p_out"])
Phx_s = qs / 3600 * wprops_s.rho * cp_s * (ds["Thx_s_out"] - Ts_in)

logger.info(f'Tp_out_min: {Tp_out_min:.2f} C, Ts_out_max: {Ts_out_max:.2f} C')
logger.info(f'Measured values, Tp_out: {ds["Thx_p_out"]:.2f} C, Ts_out: {ds["Thx_s_out"]:.2f} C')
logger.info(f'Measured power, Phx_p: {Phx_p:.2f} W, Phx_s: {Phx_s:.2f} W')

In [154]:
# Calibrate model parameters
from solarMED_modeling.calibration.parameters_fit import objective_function
from scipy.optimize import minimize

# Inputs 
Tp_in = df.iloc[idx_start:idx_end]['Thx_p_in'].values
Ts_in = df.iloc[idx_start:idx_end]['Thx_s_in'].values
qp = df.iloc[idx_start:idx_end]['qhx_p'].values
qs = df.iloc[idx_start:idx_end]['qhx_s'].values
Tamb = df.iloc[idx_start:idx_end]['Tamb'].values

# Experimental outputs
Tp_out_ref = df.iloc[idx_start:idx_end]['Thx_p_out'].values
Ts_out_ref = df.iloc[idx_start:idx_end]['Thx_s_out'].values

# Define optimizer inputs
# Tp_in:float, Ts_in:float, qp:float, qs:float, Tamb:float, UA:float=28000, H:float=0,
#                          hex_type:Literal['counter_flow', ] = 'counter_flow', return_epsilon: bool = False
inputs = [Tp_in, Ts_in, qp, qs, Tamb]  # Input values
outputs = np.column_stack((Tp_out_ref, Ts_out_ref))  # Actual output values
params_objective_function = {'metric': 'IAE', 'recursive':False, 'n_outputs':2, 
                             'n_parameters': 2} # 'len_outputs':[N, N]
params = (False, )    # Constant model parameters (log, hex_type, return_epsilon)

# Set initial parameter values
initial_parameters = [2.16e3, 0]
# initial_parameters = [0.85, 0.85]

#         beta min, beta max    Hmin,    Hmax
bounds = ((500, 100000), (0,20))
# bounds = ((0, 10), (0, 10))

# Perform parameter calibration
optimized_parameters = minimize(
    objective_function,
    initial_parameters,
    args=(heat_exchanger_model, inputs, outputs, params, params_objective_function),
    bounds = bounds,
    method= 'Nelder-Mead'#'Powell'
).x

UA = optimized_parameters[0]
H = optimized_parameters[1]

logger.info(f'Optimized parameters, UA: {UA:.2f}, H: {H:.2f}')


In [163]:
df_mod = pd.DataFrame()
df_mod_given_eps = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    
    ds = df.iloc[idx]
        
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
    
    Thx_p_in = ds['Thx_p_in']
    Thx_s_in = ds['Thx_s_in']
    qhx_p = within_range_or_zero_or_max(ds['qhx_p'], range=[5, 10])
    # qhx_s = within_range_or_zero_or_max(ds['qhx_s'], range=[0.9, 1.7])
    qhx_s = ds['qhx_s2_estimated']
    Tamb = ds['Tamb']
    
    Thx_p_out, Thx_s_out, epsilon = heat_exchanger_model(Tp_in=Thx_p_in, Ts_in=Thx_s_in, qp=qhx_p, qs=qhx_s, Tamb=Tamb, UA=UA, H=H, return_epsilon=True)
    
    try:
        estimated_epsilon = calculate_heat_transfer_effectiveness(Tp_in=Thx_p_in, Tp_out=ds["Thx_p_out"], Ts_in=Thx_s_in, Ts_out=ds["Thx_s_out"], qp=qhx_p, qs=qhx_s)
        
        df.loc[df.index[idx], 'epsilon'] = estimated_epsilon
    except:
        pass
    
    Thx_p_out2, Thx_s_out2 = heat_exchanger_model(Tp_in=Thx_p_in, Ts_in=Thx_s_in, qp=qhx_p, qs=qhx_s, Tamb=Tamb, H=0, return_epsilon=False, epsilon=estimated_epsilon)
    
    result = pd.DataFrame({
        'Thx_p_out': Thx_p_out,
        'Thx_s_out': Thx_s_out,
        'epsilon': epsilon,
        'qhx_s': qhx_s
    }, index=[0])
    
    result2 = pd.DataFrame({
        'Thx_p_out': Thx_p_out2,
        'Thx_s_out': Thx_s_out2,
        'epsilon': estimated_epsilon,
        'qhx_s': qhx_s
    }, index=[0])
    
    logger.info(f"Finished iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} s")
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)
    df_mod_given_eps = pd.concat([df_mod_given_eps, result2], ignore_index=True)
    
    
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]
df_mod_given_eps.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=[df_mod, df_mod_given_eps], vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'heat_exhanger_validation_{df.index[0].strftime("%Y%m%d")}')
)

In [165]:
# Save figure
save_figure(
    figure_name=f"heat_exchanger_validation", 
    figure_path=Path("/home/jmserrano/Downloads"),
    fig=fig, formats=('svg', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)